In [ ]:
!pip install tensorflow transformers scikit-learn pandas numpy


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving updated_dataset (1).xlsx to updated_dataset (1) (1).xlsx


In [ ]:
import pandas as pd

# Load the dataset with the correct name
df = pd.read_excel("updated_dataset (1) (1).xlsx")

# Verify if it's loaded properly
print(df.head())


       tweet_id      reply_id                 created_at  retweet_count  \
0  1.622371e+18  1.622371e+18  2023-02-05 23:06:43+00:00            6.0   
1  1.622955e+18  1.622371e+18  2023-02-07 13:47:19+00:00            0.0   
2  1.622784e+18  1.622371e+18  2023-02-07 02:27:52+00:00            0.0   
3  1.622769e+18  1.622371e+18  2023-02-07 01:26:17+00:00            1.0   
4  1.622696e+18  1.622371e+18  2023-02-06 20:37:31+00:00            1.0   

   favorite_count                                               text  \
0            45.0  Fuck this horrendous woman trying to blame Cor...   
1             0.0  Baddiel rightfully calls out bigots who masque...   
2             1.0  She's a serial liar. :Ireland::Palestinian_Ter...   
3             3.0  She is completely delusional! Is she more conc...   
4             2.0  Yes. I queried her support for Baddiel politel...   

     Insult    Threat  Identity_Attack  Profanity  ...  \
0  0.168538  0.007288         0.067497   0.018529  ...   


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Drop missing values
df = df.dropna(subset=["text", "Bystander Roles Label", "Cyberbullying label"])

# Encode Bystander Roles and Cyberbullying Labels
bystander_encoder = LabelEncoder()
df["Bystander_Role_Encoded"] = bystander_encoder.fit_transform(df["Bystander Roles Label"])

cyberbullying_encoder = LabelEncoder()
df["Cyberbullying_Label_Encoded"] = cyberbullying_encoder.fit_transform(df["Cyberbullying label"])

df[["text", "Bystander_Role_Encoded", "Cyberbullying_Label_Encoded"]].head()


<ipython-input-8-fec465295c80>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Bystander_Role_Encoded"] = bystander_encoder.fit_transform(df["Bystander Roles Label"])
<ipython-input-8-fec465295c80>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Cyberbullying_Label_Encoded"] = cyberbullying_encoder.fit_transform(df["Cyberbullying label"])


,text,Bystander_Role_Encoded,Cyberbullying_Label_Encoded
0,Fuck this horrendous woman trying to blame Cor...,0,4
1,Baddiel rightfully calls out bigots who masque...,0,0
2,She's a serial liar. :Ireland::Palestinian_Ter...,0,0
3,She is completely delusional! Is she more conc...,0,0
4,Yes. I queried her support for Baddiel politel...,0,0


In [ ]:
from sklearn.model_selection import train_test_split

# Split for Bystander Role Detection
train_texts, test_texts, train_bystander, test_bystander = train_test_split(
    df["text"].values, df["Bystander_Role_Encoded"].values, test_size=0.2, random_state=42)

test_texts, val_texts, test_bystander, val_bystander = train_test_split(
    test_texts, test_bystander, test_size=0.5, random_state=42)

train_texts.shape, val_texts.shape, test_texts.shape


((38651,), (4832,), (4831,))

In [ ]:
from transformers import BertTokenizer

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize text
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=256)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length=256)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=256)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
import tensorflow as tf

def convert_to_tf_dataset(encodings, labels):
    return tf.data.Dataset.from_tensor_slices((
        dict(encodings),
        labels
    )).batch(16)

train_dataset = convert_to_tf_dataset(train_encodings, train_bystander)
val_dataset = convert_to_tf_dataset(val_encodings, val_bystander)
test_dataset = convert_to_tf_dataset(test_encodings, test_bystander)


In [ ]:
from transformers import TFBertForSequenceClassification, AdamW

# Load BERT model for classification
bert_model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(df["Bystander_Role_Encoded"].unique()))

# Compile Model
bert_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

# Train Model
bert_model.fit(train_dataset, validation_data=val_dataset, epochs=3)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
2416/2416 [==============================] - 2091s 845ms/step - loss: 0.5279 - accuracy: 0.7866 - val_loss: 0.4531 - val_accuracy: 0.8135
Epoch 2/3
2416/2416 [==============================] - 2044s 846ms/step - loss: 0.2711 - accuracy: 0.8994 - val_loss: 0.5042 - val_accuracy: 0.8185
Epoch 3/3
2416/2416 [==============================] - 2042s 845ms/step - loss: 0.1705 - accuracy: 0.9408 - val_loss: 0.5788 - val_accuracy: 0.8179


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
bert_model.save_pretrained("/content/drive/My Drive/bystander_bert_model")
tokenizer.save_pretrained("/content/drive/My Drive/bystander_bert_model")


('/content/drive/My Drive/bystander_bert_model/tokenizer_config.json',
 '/content/drive/My Drive/bystander_bert_model/special_tokens_map.json',
 '/content/drive/My Drive/bystander_bert_model/vocab.txt',
 '/content/drive/My Drive/bystander_bert_model/added_tokens.json')

In [4]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
import os
print(os.listdir("/content/drive/MyDrive/"))


['shanis_RoleApplicationVideo.mp4', 'Shanis k.pdf (1).pdf', 'CYBY23_dataset_SVM_tuned.csv', 'Colab Notebooks', 'CYBY23_dataset_BERT_tuned.csv', 'CYBY23_dataset_BERT_tuned.xlsx', 'archive (5).zip', 'cyberbullying_tweets.csv', 'BERT_Bystander_Roles', 'Predicted_Cyberbullying_Dataset.csv', 'Integrated_Cyberbullying_Dataset.csv', 'updated_dataset (1).xlsx', 'bystander_bert_model']


In [6]:
from transformers import TFBertForSequenceClassification, BertTokenizer

# Define the model path in Google Drive
model_path = "/content/drive/MyDrive/bystander_bert_model"

# Reload the model and tokenizer
bert_model = TFBertForSequenceClassification.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained(model_path)

print("Model loaded successfully! ✅")


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at /content/drive/MyDrive/bystander_bert_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


Model loaded successfully! ✅


In [8]:
# Check for missing values
print(df["text"].isnull().sum())  # Should be 0 after fixing

# Fill NaN values with empty strings and ensure all text is string format
df["text"] = df["text"].fillna("").astype(str)


4


In [9]:
import numpy as np
import pandas as pd
from transformers import BertTokenizer

# ✅ Load the dataset from Google Drive
dataset_path = "/content/drive/MyDrive/updated_dataset (1).xlsx"
df = pd.read_excel(dataset_path)

# ✅ Ensure "cleaned_text" has no missing values
df["cleaned_text"] = df["cleaned_text"].fillna("").astype(str)

# ✅ Load the tokenizer (if not already loaded)
model_path = "/content/drive/MyDrive/bystander_bert_model"
tokenizer = BertTokenizer.from_pretrained(model_path)

# ✅ Tokenize the cleaned text for BERT
test_encodings = tokenizer(
    list(df["cleaned_text"]),  # ✅ Use "cleaned_text" instead of "text"
    truncation=True,
    padding=True,
    max_length=256,
    return_tensors="tf"
)

# ✅ Predict Bystander Roles using the already loaded BERT model
bystander_predictions = bert_model.predict(test_encodings)

# ✅ Convert logits to class labels
df["Predicted_Bystander_Role"] = np.argmax(bystander_predictions.logits, axis=1)

# ✅ Save the dataset with predictions to Google Drive
output_path = "/content/drive/MyDrive/Predicted_Cyberbullying_Dataset.csv"
df.to_csv(output_path, index=False)

print("✅ Bystander role predictions saved successfully at:", output_path)


1510/1510 [==============================] - 842s 552ms/step
✅ Bystander role predictions saved successfully at: /content/drive/MyDrive/Predicted_Cyberbullying_Dataset.csv


In [10]:
import pandas as pd

# Load the dataset with BERT predictions
dataset_path = "/content/drive/MyDrive/Predicted_Cyberbullying_Dataset.csv"
df = pd.read_csv(dataset_path)

# Show the first few rows
df.head()


<ipython-input-10-dd4c4720d9bb>:5: DtypeWarning: Columns (2,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(dataset_path)


,tweet_id,reply_id,created_at,retweet_count,favorite_count,text,Insult,Threat,Identity_Attack,Profanity,...,Cyberbullying label,Bystander Roles Label Encoded,RF_Prediction,SVM_Prediction,Final_Label,SVM_Tuned_Prediction,Cyberbullying label Encoded,BERT_Prediction,cleaned_text,Predicted_Bystander_Role
0,1.622371e+18,1.622371e+18,2023-02-05 23:06:43+00:00,6.0,45.0,Fuck this horrendous woman trying to blame Cor...,0.168538,0.007288,0.067497,0.018529,...,other_cyberbullying,0.0,0.0,0.0,2.0,0.0,2.0,1.0,fuck this horrendous woman trying to blame cor...,0
1,1.622955e+18,1.622371e+18,2023-02-07 13:47:19+00:00,0.0,0.0,Baddiel rightfully calls out bigots who masque...,0.167588,0.007625,0.082528,0.020920,...,No Cyberbullying,0.0,0.0,0.0,0.0,0.0,0.0,0.0,baddiel rightfully calls out bigots who masque...,5
2,1.622784e+18,1.622371e+18,2023-02-07 02:27:52+00:00,0.0,1.0,She's a serial liar. :Ireland::Palestinian_Ter...,0.630839,0.010085,0.058402,0.176599,...,No Cyberbullying,0.0,0.0,0.0,0.0,0.0,0.0,0.0,shes a serial liar irelandpalestinianterritori...,2
3,1.622769e+18,1.622371e+18,2023-02-07 01:26:17+00:00,1.0,3.0,She is completely delusional! Is she more conc...,0.367091,0.012648,0.074256,0.122868,...,No Cyberbullying,0.0,0.0,0.0,0.0,0.0,0.0,0.0,she is completely delusional is she more conce...,5
4,1.622696e+18,1.622371e+18,2023-02-06 20:37:31+00:00,1.0,2.0,Yes. I queried her support for Baddiel politel...,0.020837,0.007703,0.002793,0.016548,...,No Cyberbullying,0.0,0.0,0.0,0.0,0.0,0.0,0.0,yes i queried her support for baddiel politely...,0


In [11]:
from sklearn.preprocessing import LabelEncoder

# Encode Cyberbullying Labels
cyberbullying_encoder = LabelEncoder()
df["Cyberbullying_Label_Encoded"] = cyberbullying_encoder.fit_transform(df["Cyberbullying label"])

# Display the label mapping
label_mapping = dict(zip(cyberbullying_encoder.classes_, cyberbullying_encoder.transform(cyberbullying_encoder.classes_)))
print("Cyberbullying Label Encoding:", label_mapping)


Cyberbullying Label Encoding: {'No Cyberbullying': 0, 'age': 1, 'ethnicity': 2, 'gender': 3, 'other_cyberbullying': 4, 'religion': 5}


In [12]:
from sklearn.model_selection import train_test_split

# Select features (Text + Bystander Role)
X = df[["cleaned_text", "Predicted_Bystander_Role"]]
y = df["Cyberbullying_Label_Encoded"]

# Train-Test Split (80% Training, 20% Testing)
train_texts, test_texts, train_labels, test_labels = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Show dataset sizes
print("Training Samples:", len(train_texts))
print("Testing Samples:", len(test_texts))


Training Samples: 38654
Testing Samples: 9664


In [14]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# ✅ Ensure all text values are strings (Fixes the error)
train_texts["cleaned_text"] = train_texts["cleaned_text"].astype(str)
test_texts["cleaned_text"] = test_texts["cleaned_text"].astype(str)

# ✅ Initialize Tokenizer
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(train_texts["cleaned_text"])

# ✅ Convert text to sequences
train_sequences = pad_sequences(tokenizer.texts_to_sequences(train_texts["cleaned_text"]), maxlen=256)
test_sequences = pad_sequences(tokenizer.texts_to_sequences(test_texts["cleaned_text"]), maxlen=256)

# ✅ Convert Predicted Bystander Role to Numpy Arrays
train_bystander = train_texts["Predicted_Bystander_Role"].values
test_bystander = test_texts["Predicted_Bystander_Role"].values


In [15]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Concatenate

# ✅ Define Input Layers
text_input = Input(shape=(256,), name="text_input")  # Text input (padded sequences)
bystander_input = Input(shape=(1,), name="bystander_input")  # Bystander role input

# ✅ Embedding + LSTM Layer for Text
embedding_layer = Embedding(input_dim=10000, output_dim=128, input_length=256)(text_input)
lstm_layer = LSTM(64)(embedding_layer)

# ✅ Concatenate LSTM output with Bystander Role input
merged = Concatenate()([lstm_layer, bystander_input])

# ✅ Final Dense Layer for Classification
output_layer = Dense(len(label_mapping), activation="softmax")(merged)

# ✅ Compile the Model
lstm_model = Model(inputs=[text_input, bystander_input], outputs=output_layer)
lstm_model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# ✅ Train the LSTM Model
lstm_model.fit(
    [train_sequences, train_bystander],  # Inputs: Text + Bystander Role
    train_labels,  # Target: Cyberbullying Type
    epochs=5,
    batch_size=32,
    validation_data=([test_sequences, test_bystander], test_labels)
)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
1208/1208 ━━━━━━━━━━━━━━━━━━━━ 25s 16ms/step - accuracy: 0.6572 - loss: 0.8319 - val_accuracy: 0.8096 - val_loss: 0.4539
Epoch 2/5
1208/1208 ━━━━━━━━━━━━━━━━━━━━ 34s 13ms/step - accuracy: 0.8603 - loss: 0.3469 - val_accuracy: 0.8103 - val_loss: 0.4440
Epoch 3/5
1208/1208 ━━━━━━━━━━━━━━━━━━━━ 22s 14ms/step - accuracy: 0.8872 - loss: 0.2798 - val_accuracy: 0.8099 - val_loss: 0.4546
Epoch 4/5
1208/1208 ━━━━━━━━━━━━━━━━━━━━ 21s 14ms/step - accuracy: 0.9052 - loss: 0.2360 - val_accuracy: 0.8085 - val_loss: 0.4923
Epoch 5/5
1208/1208 ━━━━━━━━━━━━━━━━━━━━ 20s 13ms/step - accuracy: 0.9160 - loss: 0.2018 - val_accuracy: 0.8098 - val_loss: 0.5262


In [16]:
from sklearn.metrics import classification_report

# ✅ Predict on Test Set
y_pred = lstm_model.predict([test_sequences, test_bystander])
y_pred_classes = y_pred.argmax(axis=1)  # Convert softmax output to class labels

# ✅ Print Classification Report
print(classification_report(test_labels, y_pred_classes))


302/302 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step
              precision    recall  f1-score   support

           0       0.61      0.46      0.53      1736
           1       0.96      0.97      0.97      1588
           2       0.96      0.98      0.97      1573
           3       0.86      0.85      0.85      1537
           4       0.56      0.69      0.62      1640
           5       0.93      0.95      0.94      1590

    accuracy                           0.81      9664
   macro avg       0.81      0.82      0.81      9664
weighted avg       0.81      0.81      0.81      9664



In [17]:
from imblearn.over_sampling import SMOTE

# Convert text sequences and labels into arrays
X_train_combined = np.column_stack((train_sequences, train_bystander))  # Combine text & bystander roles
y_train = train_labels.values

# Apply SMOTE to balance dataset
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_combined, y_train)

print("✅ Dataset balanced using SMOTE! New sample distribution:")
print(pd.Series(y_train_balanced).value_counts())


✅ Dataset balanced using SMOTE! New sample distribution:
1    6763
0    6763
5    6763
4    6763
2    6763
3    6763
Name: count, dtype: int64


In [18]:
# Retrain the LSTM model with balanced data
lstm_model.fit(
    [X_train_balanced[:, :-1], X_train_balanced[:, -1]],  # Split text & bystander role
    y_train_balanced,
    epochs=10,  # Increased from 5 to 10
    batch_size=32,
    validation_data=([test_sequences, test_bystander], test_labels)
)


Epoch 1/10
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 24s 18ms/step - accuracy: 0.8868 - loss: 0.3412 - val_accuracy: 0.8043 - val_loss: 0.5566
Epoch 2/10
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 27s 21ms/step - accuracy: 0.9050 - loss: 0.2483 - val_accuracy: 0.8037 - val_loss: 0.5652
Epoch 3/10
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 30s 13ms/step - accuracy: 0.9154 - loss: 0.2077 - val_accuracy: 0.7905 - val_loss: 0.6546
Epoch 4/10
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 20s 13ms/step - accuracy: 0.9255 - loss: 0.1754 - val_accuracy: 0.7912 - val_loss: 0.7027
Epoch 5/10
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 22s 14ms/step - accuracy: 0.9329 - loss: 0.1516 - val_accuracy: 0.7884 - val_loss: 0.7499
Epoch 6/10
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 21s 14ms/step - accuracy: 0.9388 - loss: 0.1389 - val_accuracy: 0.7886 - val_loss: 0.8492
Epoch 7/10
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 19s 13ms/step - accuracy: 0.9455 - loss: 0.1167 - val_accuracy: 0.7826 - val_loss: 0.9083
Epoch 8/10
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 20s 13ms/step - accuracy: 0.9499 -

In [20]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Concatenate, Bidirectional, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

# ✅ Define Input Layers
text_input = Input(shape=(256,), name="text_input")  # Text input (padded sequences)
bystander_input = Input(shape=(1,), name="bystander_input")  # Bystander role input

# ✅ Embedding + BiLSTM Layer with Dropout
embedding_layer = Embedding(input_dim=10000, output_dim=128, input_length=256)(text_input)
lstm_layer = Bidirectional(LSTM(64, return_sequences=True))(embedding_layer)
dropout_layer = Dropout(0.3)(lstm_layer)  # Dropout Layer (Prevents Overfitting)
lstm_layer_final = Bidirectional(LSTM(64))(dropout_layer)

# ✅ Concatenate LSTM output with Bystander Role input
merged = Concatenate()([lstm_layer_final, bystander_input])

# ✅ Final Dense Layer for Classification
output_layer = Dense(len(label_mapping), activation="softmax")(merged)

# ✅ Compile the Model with Lower Learning Rate
lstm_model = Model(inputs=[text_input, bystander_input], outputs=output_layer)
lstm_model.compile(
    optimizer=Adam(learning_rate=1e-4),  # ✅ Lower Learning Rate for Stable Training
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# ✅ Apply Early Stopping to Prevent Overfitting
early_stop = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

# ✅ Train the Improved LSTM Model
lstm_model.fit(
    [X_train_balanced[:, :-1], X_train_balanced[:, -1]],  # Inputs: Text + Bystander Role
    y_train_balanced,  # Target: Cyberbullying Type
    epochs=20,  # More epochs, but stops early if needed
    batch_size=32,
    validation_data=([test_sequences, test_bystander], test_labels),
    callbacks=[early_stop]  # ✅ Stops training when validation accuracy stops improving
)

print("✅ Training Complete! Model is now optimized for better real-world performance.")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/20
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 60s 42ms/step - accuracy: 0.3779 - loss: 1.4247 - val_accuracy: 0.7576 - val_loss: 0.5771
Epoch 2/20
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 78s 40ms/step - accuracy: 0.7546 - loss: 0.6121 - val_accuracy: 0.7860 - val_loss: 0.5133
Epoch 3/20
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 50s 40ms/step - accuracy: 0.7963 - loss: 0.5268 - val_accuracy: 0.8001 - val_loss: 0.4797
Epoch 4/20
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 81s 39ms/step - accuracy: 0.8195 - loss: 0.4768 - val_accuracy: 0.8069 - val_loss: 0.4726
Epoch 5/20
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 49s 39ms/step - accuracy: 0.8413 - loss: 0.4336 - val_accuracy: 0.8035 - val_loss: 0.4761
Epoch 6/20
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 83s 40ms/step - accuracy: 0.8529 - loss: 0.4065 - val_accuracy: 0.8039 - val_loss: 0.4797
Epoch 7/20
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 49s 39ms/step - accuracy: 0.8634 - loss: 0.3873 - val_accuracy: 0.8019 - val_loss: 0.4971
✅ Training Complete! Model is now optimized for better real-world performanc

In [22]:
# ✅ Save the model with the correct format
model_path = "/content/drive/MyDrive/cyberbullying_lstm_model.keras"  # Add .keras extension
lstm_model.save(model_path)

print("✅ Model saved successfully at:", model_path)


✅ Model saved successfully at: /content/drive/MyDrive/cyberbullying_lstm_model.keras
